<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/main/2025/handson/CurveFittingPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curve Fitting in Python with `scipy.optimize.curve_fit`

Read: [SciPy CurveFit documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)


import the necessary libraries:

- `numpy` for numerical operations,
- `matplotlib.pyplot` for plotting data,
- `scipy.optimize.curve_fit` for fitting curves.

In [ ]:
# Run this cell to import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

## Example 1 - Constant Function

#### Physics Context:
Imagine you're measuring the voltage of a battery using a voltmeter. The voltage should ideally remain constant for reasonable amounts of time.

The equation for a constant function is:
$$ y = c $$

Where:
- $ c $ is the constant value.

We'll simulate some data with random noise and use `curve_fit` to determine the value of $ c $.


In [ ]:
# Generate synthetic data
time = np.linspace(0, 10, 20)  # Time in seconds
true_c = 5.0  # True constant voltage in volts
voltage = true_c + np.random.normal(0, 0.2, size=len(time))  # Add noise

# Define the constant function
def fit_constant_func(x, c):
    """ fitting function for a constant value independent from time """
    return c

# Use curve_fit to find the best-fit parameter
params, covariance = curve_fit(fit_constant_func, time, voltage)
fitted_c = params[0]

# Plot the data and fitted curve
plt.scatter(time, voltage, label="Data (with noise)", color="blue")
plt.axhline(fitted_c, color="red", linestyle="--", label= "Fitted c = {:.2f}".format(fitted_c))
plt.xlabel("Time (s)")
plt.ylabel("Voltage (V)")
plt.ylim(0, 7)  # Set the y-axis limits
plt.title("Constant Function Fit")
plt.legend()
plt.show()

## Example 2 - Linear Function

#### Physics Context:
Hooke's Law states that the force exerted by a spring is proportional to its displacement:
$$ F = kx $$

Where:
- $ F $ is the force,
- $ k $ is the spring constant,
- $ x $ is the displacement.

This is a linear relationship. Let's simulate some data and fit it using `curve_fit`.

In [ ]:
# Generate synthetic data
displacement = np.linspace(0, 10, 20)  # Displacement in meters
true_k = 3.0  # True spring constant in N/m
force = true_k * displacement + np.random.normal(0, 1.0, size=len(displacement))  # Add noise

# Define the linear function
def fit_linear_func(x, m):
    """ fitting function for a linear realtionship """
    return m * x

# Use curve_fit to find the best-fit parameter
params, covariance = curve_fit(fit_linear_func, displacement, force)
fitted_k = params[0]

# Plot the data and fitted curve
plt.scatter(displacement, force, label="Data (with noise)", color="blue")
plt.plot(displacement, fit_linear_func(displacement, fitted_k), color="red", linestyle="--",
         label="Fitted k = {:.2f} N/m".format(fitted_k))
plt.xlabel("Displacement (m)")
plt.ylabel("Force (N)")
plt.title("Linear Function Fit")
plt.legend()
plt.show()

## Example 3 - Non-Linear Function

#### Physics Context:
Radioactive decay follows an exponential decay law:
$$ N(t) = N_0 e^{-\lambda t} $$

Where:
- $ N(t) $ is the number of radioactive nuclei at time $ t $,
- $ N_0 $ is the initial number of nuclei,
- $ \lambda $ is the decay constant.

In [ ]:
# Generate synthetic data
time = np.linspace(0, 10, 20)  # Time in seconds
N_0_true = 100.0  # Initial number of nuclei
lambda_true = 0.3  # Decay constant in s^-1
N_t = N_0_true * np.exp(-lambda_true * time) + np.random.normal(0, 5.0, size=len(time))  # Add noise

# Define the exponential decay function
def fit_exp_decay(t, N_0, lambda_):
    """ fitting function for an exponential decay """
    return N_0 * np.exp(-lambda_ * t)

# Use curve_fit to find the best-fit parameters
params, covariance = curve_fit(fit_exp_decay, time, N_t)
fitted_N_0, fitted_lambda = params

# Plot the data and fitted curve
plt.scatter(time, N_t, label="Data (with noise)", color="blue")
plt.plot(time, fit_exp_decay(time, fitted_N_0, fitted_lambda), color="red", linestyle="--",
         label=f"Fitted N_0 = {fitted_N_0:.2f}, λ = {fitted_lambda:.2f}")
plt.xlabel("Time (s)")
plt.ylabel("Number of Nuclei")
plt.title("Exponential Decay Fit")
plt.legend()
plt.show()

## Example 4: Fitting a spectral line - Use a Gaussian profile to model an emission line in a spectrum. This is a common task in astronomical spectroscopy.

**Stop here: Think about why you would fit a gaussian!!!**

![California Nebula NGC1499](https://github.com/ubsuny/PHY386/raw/main/media/CaliforniaNebulaNGC1499_2025-03-10.JPG "California Nebula NGC1499")

This example demonstrates fitting a Gaussian profile to a simulated spectral line, which could represent the [H-alpha emission line](https://en.wikipedia.org/wiki/Hydrogen-alpha) at 6563 Å as observed for example in in [NGC1499](https://en.wikipedia.org/wiki/California_Nebula). The code generates synthetic data with added noise, then uses `scipy.optimize.curve_fit` to fit a Gaussian function to this data.

In [ ]:
# Generate synthetic spectral data
wavelength = np.linspace(6550, 6580, 100)  # Wavelength in Angstroms
true_amplitude = 10.0
true_center = 6563.0  # H-alpha line
true_sigma = 2.0

# Gaussian function to model the spectral line
def fit_gaussian(x, amplitude, center, sigma):
    """ fitting function for a single gaussian function """
    return amplitude * np.exp(-(x - center)**2 / (2 * sigma**2))

# Create synthetic spectrum with noise
flux = fit_gaussian(wavelength, true_amplitude, true_center, true_sigma)
noise = np.random.normal(0, 0.5, wavelength.shape)
noisy_gaussian_flux = flux + noise

plt.figure(figsize=(10, 6))
plt.plot(wavelength, noisy_gaussian_flux, 'b.', label='Noisy data')
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux')
plt.legend()


In [ ]:
# Fit the noisy data
popt, pcov = curve_fit(fit_gaussian, wavelength, noisy_gaussian_flux)

# Extract fitted parameters
fitted_amplitude, fitted_center, fitted_sigma = popt

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(wavelength, noisy_gaussian_flux, 'b.', label='Noisy data')
plt.plot(wavelength, fit_gaussian(wavelength, *popt), 'r-', label='Fitted curve')
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux')
plt.title('Fitting a Gaussian to a Spectral Line')
plt.legend()

print(f"Fitted parameters:")
print(f"Amplitude: {fitted_amplitude:.2f}")
print(f"Center: {fitted_center:.2f} Å")
print(f"Sigma: {fitted_sigma:.2f} Å")

plt.show()

That is dissapointing!

**For more complex functions we have to provide an intial guess**

In [ ]:
# Fit the noisy data
initial_guess = [8, 6565, 1.5]  # Initial guesses for [amplitude, center, sigma]
popt, pcov = curve_fit(fit_gaussian, wavelength, noisy_gaussian_flux, p0=initial_guess)

# Extract fitted parameters
fitted_amplitude, fitted_center, fitted_sigma = popt

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(wavelength, noisy_gaussian_flux, 'b.', label='Noisy data')
plt.plot(wavelength, fit_gaussian(wavelength, *popt), 'r-', label='Fitted curve')
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux')
plt.title('Fitting a Gaussian to a Spectral Line')
plt.legend()

print(f"Fitted parameters:")
print(f"Amplitude: {fitted_amplitude:.2f}")
print(f"Center: {fitted_center:.2f} Å")
print(f"Sigma: {fitted_sigma:.2f} Å")

plt.show()

## Example 5: Let's modify example 5 to fit a **Lorentzian function** instead of a Gaussian.

This approach is particularly useful when modeling spectral lines dominated by the natural line width. If Doppler broadening is significant (e.g., due to thermal motion), you might consider fitting the **Gaussian** above or even a **Voigt profile**, which combines Gaussian and Lorentzian components.

1. **Lorentzian Function**:
   The Lorentzian profile is defined as:
   $$
   L(x; A, x_0, \gamma) = A \cdot \frac{\gamma^2}{(x - x_0)^2 + \gamma^2}
   $$
   Where:
   - $ A $: Amplitude of the peak.
   - $ x_0 $: The center of the peak (e.g., the central wavelength of the spectral line).
   - $ \gamma $: Half-width at half-maximum (HWHM), which describes the width of the peak.

2. **Synthetic Data**:
   We simulate a spectral line centered at $ 6563 $ Å with an amplitude of $ 10 $ and HWHM ($ \gamma $) of $ 1.5 $. Random Gaussian noise is added to simulate real-world measurement uncertainties.

3. **Curve Fitting**:
   The `curve_fit` function from `scipy.optimize` is used to fit the noisy data to the Lorentzian model. We provide an initial guess for the parameters: amplitude ($ A $), center ($ x_0 $), and HWHM ($ \gamma $).





In [ ]:
# Generate synthetic spectral data
wavelength = np.linspace(6550, 6580, 100)  # Wavelength in Angstroms
true_amplitude = 10.0
true_center = 6563.0  # H-alpha line center
true_gamma = 1.5  # Lorentzian half-width at half-maximum (HWHM)

# Define the Lorentzian function
def fit_lorentzian(x, amplitude, center, gamma):
    """ fitting function for a single lorentzian function """
    return amplitude * (gamma**2 / ((x - center)**2 + gamma**2))

# Create synthetic spectrum with noise
flux = fit_lorentzian(wavelength, true_amplitude, true_center, true_gamma)
noise = np.random.normal(0, 0.5, wavelength.shape)  # Add random noise
noisy_lorentzian_flux = flux + noise

plt.figure(figsize=(10, 6))
plt.plot(wavelength, noisy_lorentzian_flux, 'b.', label='Noisy data')
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux')
plt.legend()


In [ ]:
# Fit the noisy data using curve_fit
initial_guess = [8, 6565, 1.0]  # Initial guesses for [amplitude, center, gamma]
popt, pcov = curve_fit(fit_lorentzian, wavelength, noisy_lorentzian_flux, p0=initial_guess)

# Extract fitted parameters
fitted_amplitude, fitted_center, fitted_gamma = popt

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(wavelength, noisy_lorentzian_flux, 'b.', label='Noisy data')
plt.plot(wavelength, fit_lorentzian(wavelength, *popt), 'r-', label='Fitted Lorentzian')
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux')
plt.title('Fitting a Lorentzian to a Spectral Line')
plt.legend()

# Print fitted parameters
print(f"Fitted parameters:")
print(f"Amplitude: {fitted_amplitude:.2f}")
print(f"Center: {fitted_center:.2f} Å")
print(f"Gamma (HWHM): {fitted_gamma:.2f} Å")

plt.show()

In [ ]:
# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(wavelength, fit_gaussian(wavelength, *popt), 'b-', label='Fitted Gaussian')
plt.plot(wavelength, fit_lorentzian(wavelength, *popt), 'k-', label='Fitted Lorentzian')
plt.xlabel('Wavelength (Å)')
plt.ylabel('Flux')
plt.title('Fitting a Gaussian to a Spectral Line')
plt.legend()

## Determining the 'goodness' of a fit: the $R^2$ value

To calculate the $$ R^2 $$ value (coefficient of determination) for a curve fit, you can use the following equation:

$$
R^2 = 1 - \frac{\text{SQ}_{\text{res}}}{\text{SQ}_{\text{tot}}}
$$

Where:
- $ \text{SQ}_{\text{res}} $ is the residual sum of squares: the sum of squared differences between the observed data and the predicted data.
- $ \text{SQ}_{\text{tot}} $ is the total sum of squares: the sum of squared differences between the observed data and its mean.

In [ ]:
def calculate_r_squared(y_observed, y_predicted):
    """
    Calculate the R^2 (coefficient of determination) value.

    Parameters:
        y_observed (array-like): The observed data points.
        y_predicted (array-like): The predicted data points from the fit.

    Returns:
        The R^2 value.
    """
    # Residual sum of squares
    sq_res = np.sum((y_observed - y_predicted)**2)

    # Total sum of squares
    sq_tot = np.sum((y_observed - np.mean(y_observed))**2)

    # R-squared
    r_squared = 1 - (sq_res / sq_tot)

    return r_squared

In [ ]:
# Calculate R^2
r_squared = calculate_r_squared(noisy_lorentzian_flux, fit_lorentzian(wavelength, *popt))
print(f"R^2 value: {r_squared:.4f}")